<br>

In [ ]:
#| include: false
from fastai.vision.all import *
from fasterai.misc.all import *

import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Get the data

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

## 2. Train the model

In [ ]:
learn = Learner(dls, vgg16_bn(num_classes=2), metrics=accuracy)
learn.fit_one_cycle(3)

3. Decompose !

In [ ]:
#| include: false
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
fc = FC_Decomposer()
new_model = fc.decompose(learn.model)

The fc layers have been factorized and replace by smaller ones.

In [ ]:
new_model

We can see compare the amount of parameters before/after:

In [ ]:
count_parameters(learn.model)

In [ ]:
count_parameters(new_model)

This represents a decrease of ~40M parameters ! 

Now this is an approximation, so it isn't really lossless and we should expect to see a performance drop, which will be bigger as we keep fewer singular values. Here we have:

In [ ]:
new_learn = Learner(dls, new_model, metrics=accuracy)
new_learn.validate()